In this notebook we compare the two methods

    *FTS Stokesian dynamics via a direct solver
    *PyStokes: superposition approximation
    
for two spheres falling in a direction perpendicular to their line of centres due to gravity. We shall look at velocity and angular velocity. Both are assumed to be zero at the start of the fall. 

Here, we want to investigate whether we can match PyStokes and FTS by considering the factors beyond superposition approximation.

<img src="./images/IMG_1346.jpeg" style="width: 500px;"/>

Explicitly, these are 

    *G1s2s * F2s
    *G2a2s * F2s
    
for the velocity and angular velocity, respectively. 

In [1]:
from ftssd import FTS
import autograd.numpy as np, matplotlib.pyplot as plt

<font size="5">FTS Stokesian dynamics

In [2]:
# particle radius, self-propulsion speed, number and fluid viscosity
b, Np, eta = 0.5, 2, 0.1
fts = FTS(b, Np, eta)

In [3]:
%%time
## Two spheres falling next to each other: FTS

#initial position
r = np.array([0.,2.3, 0.,0., 100.,100.])

#gravity in neg z direction
F = np.array([0.,0., 0.,0., -0.,-1.])
T = np.array([0.,0., 0.,1.,  0., 0.])  ##torque

# integration parameters
Nt=2; r1=np.zeros([3,Nt]); r2=np.zeros([3,Nt])
r1[:,0] = r[::Np]; r2[:,0] = r[1::Np]
dt=3.;
v = np.zeros(3*Np);
o = np.zeros(3*Np)
theta1=np.zeros([3,Nt]); theta2=np.zeros([3,Nt])
v1 = np.zeros([3,Nt-1]); v2 = np.zeros([3,Nt-1])
o1 = np.zeros([3,Nt-1]); o2 = np.zeros([3,Nt-1])

for i in range(Nt-1):
    fts.directSolve(v, o, r, F, T)
    
    r1[:,i+1] = r1[:,i] + dt*v[::Np]
    r2[:,i+1] = r2[:,i] + dt*v[1::Np]
    
    theta1[:,i+1] = theta1[:,i] + dt*o[::Np]
    theta2[:,i+1] = theta2[:,i] + dt*o[1::Np]
    
    #get velocity (constant for this case)
    v1[:,i] = v[::Np]
    v2[:,i] = v[1::Np]
    
    #get angular velocity (constant for this case)
    o1[:,i] = o[::Np]
    o2[:,i] = o[1::Np]
    
    # reset variables for next time step
    r[::Np] = r1[:,i+1]
    r[1::Np] = r2[:,i+1]
    v = v*0
    o = o*0

print('finish')

finish
CPU times: user 4.25 s, sys: 48.5 ms, total: 4.3 s
Wall time: 4.27 s


In [4]:
## velocity and angular velocity of particle 1
v1[:,0], o1[:,0]

(array([ 0.        ,  0.        , -0.10322986]),
 array([0.        , 0.05886391, 0.        ]))

In [5]:
## velocity and angular velocity of particle 2
v2[:,0], o2[:,0]

(array([ 0.        ,  0.        , -1.06695673]),
 array([0.        , 3.13200627, 0.        ]))

--------------------------------

<font size="5">PyStokes (superposition approximation)

In [6]:
import pystokes

In [7]:
pstk = pystokes.unbounded.Rbm(b, Np, eta)

In [8]:
%%time
## Two spheres falling next to each other: pystokes

#initial position
r = np.array([0.,2.3, 0.,0., 100.,100.])

#gravity in neg z direction
F = np.array([0.,0., 0.,0., -0.,-1.])
T = np.array([0.,0., 0.,1.,  0., 0.])  ##torque

# integration parameters
Nt=2; r1=np.zeros([3,Nt]); r2=np.zeros([3,Nt])
r1[:,0] = r[::Np]; r2[:,0] = r[1::Np]
dt=3.;
v = np.zeros(3*Np);
o = np.zeros(3*Np)
theta1=np.zeros([3,Nt]); theta2=np.zeros([3,Nt])
v1_pstk = np.zeros([3,Nt-1]); v2_pstk = np.zeros([3,Nt-1])
o1_pstk = np.zeros([3,Nt-1]); o2_pstk = np.zeros([3,Nt-1])

for i in range(Nt-1):
    pstk.mobilityTT(v, r, F)
    pstk.mobilityTR(v, r, T)
    pstk.mobilityRT(o, r, F)
    pstk.mobilityRR(o, r, T)
    
    r1[:,i+1] = r1[:,i] + dt*v[::Np]
    r2[:,i+1] = r2[:,i] + dt*v[1::Np]
    
    theta1[:,i+1] = theta1[:,i] + dt*o[::Np]
    theta2[:,i+1] = theta2[:,i] + dt*o[1::Np]
    
    #get velocity (constant for this case)
    v1_pstk[:,i] = v[::Np]
    v2_pstk[:,i] = v[1::Np]
    
    #get angular velocity (constant for this case)
    o1_pstk[:,i] = o[::Np]
    o2_pstk[:,i] = o[1::Np]
    
    # reset variables for next time step
    r[::Np] = r1[:,i+1]
    r[1::Np] = r2[:,i+1]
    v = v*0
    o = o*0

print('finish')

finish
CPU times: user 658 µs, sys: 599 µs, total: 1.26 ms
Wall time: 711 µs


In [9]:
## velocity and angular velocity of particle 1
v1_pstk[:,0], o1_pstk[:,0]

(array([ 0.        ,  0.        , -0.10322986]),
 array([0.        , 0.05886391, 0.        ]))

In [10]:
## velocity and angular velocity of particle 2
v2_pstk[:,0], o2_pstk[:,0]

(array([ 0.        ,  0.        , -1.06103295]),
 array([0.        , 3.18309886, 0.        ]))

-----------------------------

Aside: In free space, for two bodies, the first order in the Jacobi iteration is the same as the zeroth order (=superposition approximation), so there will be no correction. Does that seem right?

Corrections appear at three bodies or when there are boundaries.

---------------------------------


<font size="5">Do they match?

Compute the contribution we get from G1s2s F2s

In [11]:
r = np.array([0.,2.3, 0.,0., 100.,100.])

#gravity in neg z direction
F = np.array([0.,0., 0.,0., -0.,-1.])
T = np.array([0.,0., 0.,1.,  0., 0.])  ##torque

i=0; j=1

xij = r[i]    - r[j]
yij = r[i+Np]  - r[j+Np]
zij = r[i+2*Np]  - r[j+2*Np]

force  = np.array([F[j],F[j+Np], F[j+2*Np]])
torque = np.array([T[j],T[j+Np], T[j+2*Np]])

lhs = fts.tensorG2s2s(xij,yij,zij)
lhs_mat = np.reshape(lhs, (9,9))
lhs_mat_inv = np.linalg.pinv(lhs_mat)
lhs_inv = np.reshape(lhs_mat_inv, (3,3,3,3))
rhs = np.zeros([3,3])
for k in range(Np):
    xjk = r[j]    - r[k]
    yjk = r[j+Np]  - r[k+Np]
    zjk = r[j+2*Np]  - r[k+2*Np]
    if k!=j:
        force_k  = np.array([F[k],F[k+Np], F[k+2*Np]])
        torque_k = np.array([T[k],T[k+Np], T[k+2*Np]])
                            
        rhs += (np.dot(fts.tensorG2s1s(xjk,yjk,zjk), force_k) 
                + 1./b * np.dot(fts.tensorG2s2a(xjk,yjk,zjk), torque_k))
    else:
        pass #otherwise have diagonal elements here
F2s = np.einsum('ijkl, kl', lhs_inv, rhs)

In [12]:
F2s

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

See whether if we add this to v and o from pystokes with G1s2s and G2a2s it becomes the same. 

In [13]:
v_ = - np.einsum('ijk,jk',fts.tensorG1s2s(xij,yij,zij),F2s)
v_

array([-0., -0., -0.])

In [14]:
## velocity difference between FTS and pystokes
v1 - v1_pstk

array([[0.],
       [0.],
       [0.]])

Indeed, the difference between the velocities of FTS and PyStokes is equal to G1s2s * F2s.

________________________________


Now compare the angular velocity

In [15]:
o_ = - np.einsum('ijk,jk',fts.tensorG2a2s(xij,yij,zij),F2s)
o_

array([-0., -0., -0.])

In [16]:
## angular velocity difference between FTS and pystokes
o1 - o1_pstk

array([[0.],
       [0.],
       [0.]])

Again, the difference between the angular velocities of FTS and PyStokes correctly equals G2a2s * F2s appearing in the equation for the angular velocity. 

_____________________

Here are the matrix elements that have been used for FTS

<img src="./images/IMG_1347.jpeg" style="width: 500px;"/>

<img src="./images/IMG_1348.jpeg" style="width: 500px;"/>

_________
__________

<font size="5">IGNORE: finding bugs in the code (FIXED)

___________________

Test muTR and muRR of pystokes vs our expressions

In [17]:
r

array([  0. ,   2.3,   0. ,   0. , 100. , 100. ])

In [18]:
F = np.array([0.,0., 0.,0., -0.,-1.])
T = np.array([0.,0., 0.,1.,  0., 0.])  ##torque

v = np.zeros(3*Np);
o = np.zeros(3*Np)

pstk.mobilityTR(v, r, T)
v

array([0.      , 0.      , 0.      , 0.      , 0.075215, 0.      ])

In [19]:
v = np.zeros(3*Np);
o = np.zeros(3*Np)

pstk.mobilityRR(o, r, T)
o

array([ 0.        ,  0.        , -0.01635109,  3.18309886,  0.        ,
        0.        ])

In [20]:
i=0; j=1

xij = r[i]    - r[j]
yij = r[i+Np]  - r[j+Np]
zij = r[i+2*Np]  - r[j+2*Np]

force  = np.array([F[j],F[j+Np], F[j+2*Np]])
torque = np.array([T[j],T[j+Np], T[j+2*Np]])

In [21]:
v_ = 1./b * np.dot(fts.tensorG1s2a(xij,yij,zij), torque)
v_

array([0.      , 0.      , 0.075215])

In [23]:
o_ = 1./b * np.dot(fts.tensorG2a2a(xij,yij,zij), torque)
o_

array([ 0.        , -0.01635109,  0.        ])

muRR do not match FTS tensors G2a2a. Check whether muRR is computed correctly, then check muRT and muTT as well.

Test whether curl curl G == lap G

In [24]:
from autograd import grad

In [25]:
PI = 3.14159265359

def G(xij,yij,zij, alpha,beta): #G_alpha beta
    rij = np.array([xij,yij,zij])
    r = np.linalg.norm(rij)
    return ((np.identity(3)/r + np.outer(rij,rij)/r**3)/(8*eta*PI))[alpha,beta]

def epsilon(i,j,k):
    return (i-j)*(j-k)*(k-i)/2.

def delG(xij,yij,zij, alpha,beta,gamma):
    return grad(G,gamma)(xij,yij,zij, alpha,beta)

def curlcurlG(xij,yij,zij, alpha,beta):
    curlcurlg=0
    for nu in range(3):
        for eta in range(3):
            for n in range(3):
                for m in range(3):
                    curlcurlg += (epsilon(alpha,nu,eta)*epsilon(beta,n,m)
                                  *grad(delG, nu)(xij,yij,zij, eta,m,n))
    return curlcurlg

def lapG(xij,yij,zij, alpha,beta): # nabla^2 G_alpha beta
    rij = np.array([xij,yij,zij])
    r = np.linalg.norm(rij)
    return ((np.identity(3)/r**3 - 3*np.outer(rij,rij)/r**5)/(4*eta*PI))[alpha,
                                                                       beta]

def tensorCCG(xij,yij,zij):
    g = np.zeros([3,3])
    for alpha in range(3):
        for beta in range(3):
            g[alpha,beta]=curlcurlG(xij,yij,zij, alpha,beta)
    return g

def tensorlapG(xij,yij,zij):
    g = np.zeros([3,3])
    for alpha in range(3):
        for beta in range(3):
            g[alpha,beta]=lapG(xij,yij,zij, alpha,beta)
    return g

In [26]:
np.allclose(tensorCCG(xij,yij,zij), tensorlapG(xij,yij,zij))

True

Test the same with muTR and muTT

In [27]:
F = np.array([0.,0., 0.,0., -1.,-1.])
T = np.array([0.,0., 1.,1.,  0., 0.])  ##torque

v = np.zeros(3*Np);
o = np.zeros(3*Np)

pstk.mobilityTT(v, r, F)
v

array([ 0.        ,  0.        ,  0.        ,  0.        , -1.23947782,
       -1.23947782])

In [28]:
v = np.zeros(3*Np);
o = np.zeros(3*Np)

pstk.mobilityRT(o, r, F)
o

array([ 0.      ,  0.      ,  0.075215, -0.075215,  0.      ,  0.      ])

In [29]:
i=0; j=1

xij = r[i]    - r[j]
yij = r[i+Np]  - r[j+Np]
zij = r[i+2*Np]  - r[j+2*Np]

force  = np.array([F[j],F[j+Np], F[j+2*Np]])
torque = np.array([T[j],T[j+Np], T[j+2*Np]])

In [31]:
v_ = np.dot(fts.tensorG1s1s(xij,yij,zij), force)
v_ += fts.G01s*force
v_

array([ 0.        ,  0.        , -1.23947782])

In [32]:
o_ = np.dot(fts.tensorG2a1s(xij,yij,zij), force)
o_

array([0.      , 0.075215, 0.      ])

muRT, muTT seem to work --> something is dodgy with muTR and muRR. Check functions G1s2a and G2a2a.

In [33]:
fts.tensorG1s2a(xij,yij,zij)

array([[-0.       , -0.       , -0.       ],
       [-0.       , -0.       , -0.0376075],
       [-0.       ,  0.0376075, -0.       ]])

In [34]:
def G1s2a(xij,yij,zij, alpha,beta):
    g1s2a=0.
    for nu in range(3):
        for eta in range(3):
            g1s2a += epsilon(beta,nu,eta)*delG(xij,yij,zij,alpha,eta,nu)
    return -0.5*b*g1s2a

def tensorG1s2a(xij,yij,zij):
    g=np.zeros([3,3])
    for alpha in range(3):
        for beta in range(3):
            g[alpha,beta]=G1s2a(xij,yij,zij, alpha,beta)
    return g


tensorG1s2a(xij,yij,zij)

array([[-0.       , -0.       , -0.       ],
       [-0.       , -0.       , -0.0376075],
       [-0.       ,  0.0376075, -0.       ]])


___________
Summary: muTR in PyStokes had a wrong sign, while muRT was correct. In the many-body case this is slightly subtle, see below

<img src="./images/IMG_1352.jpeg" style="width: 500px;"/>


___________